### 피아노 Midi 사이트
- https://www.freepianotutorials.net/2023/12/ludwig-goransson-can-you-hear-music.html#more

### Ⅰ. 라이브러리

In [1]:
import os
import mido
import numpy as np
import pandas as pd

from mido import MidiFile, MidiTrack, MetaMessage, Message

In [2]:
data_folder = "midi_data"

In [3]:
def load_midi_data(input_name, target_name):
    input_path  = os.path.join(data_folder, input_name)
    target_path = os.path.join(data_folder, target_name)
    
    input_mid  = mido.MidiFile(input_path)
    target_mid = mido.MidiFile(target_path)
    
    return input_mid, target_mid

In [57]:
#input_name = "Fugue3.mid"
input_name = "Fugue3.mid"
target_name = "Fugue1.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

In [4]:
b_input_mid, b_target_mid = load_midi_data("bach_846.mid", "bach_847.mid")

In [5]:
for track in b_input_mid.tracks:
    for msg in track:
        if msg.type == 'control_change':
            print(msg)

control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=127 time=0
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change channel=0 control=91 value=127 time=119
control_change cha

In [67]:
cur_time = 0
for i, msg in enumerate(input_mid.tracks[1]):
    cur_time+=msg.time
    if msg.time>0:
        seconds = mido.tick2second(cur_time, input_mid.ticks_per_beat, 731707)
        print(seconds)

print("===========")
print(cur_time)

1.0975605
1.463414
1.64634075
1.8292675
2.01219425
2.195121
2.5609745
2.926828
3.2926815
3.47560825
3.658535
4.0243885
4.390242
4.7560955
5.121949
5.4878025
5.853656
6.2195095
6.40243625
6.585363
6.76828975
6.9512165
7.13414325
7.31707
7.49999675
7.6829235
7.86585025
8.048777
8.23170375
8.4146305
8.59755725
8.780484
9.69511775
9.8780445
10.06097125
10.243898
10.42682475
10.6097515
10.79267825
10.975605
11.15853175
11.3414585
11.52438525
11.707312
11.798775375
12.0731655
12.164628875
12.439019
13.902433
15.365847
16.4634075
16.829261
17.1951145
17.9268215
17.948162954166666
18.292675
19.359747708333334
19.3902355
19.57316225
19.756089
20.823161708333334
20.8536495
21.219503
22.25608791666667
22.3170635
22.49999025
22.682917
23.749989708333334
23.7804775
24.146331
25.06096475
25.2438915
25.42681825
25.609745
26.52437875
26.7073055
26.89023225
27.073159
27.25608575
27.4390125
27.62193925
27.804866
27.98779275
28.1707195
28.35364625
28.536573
28.9024265
29.26828
29.6341335
29.81706025
29.9

### Ⅱ. Data Extraction

#### 1. note (음정) : [ 0 to 127 ]
- msg.type = 'note_on' / 'note_off'
    + 소리의 강도 / 세기 ( velocity )
    + 지속 시간 ( time, 단위 : tick )
    
- note_on / note_off 구분 X
    + note_on channel=0 note=66 velocity=64 time=0
    + note_off channel=0 note=66 velocity=48 time=60
        - note_off 도 (( note / velocity )) 존재

In [5]:
def distribute_msg(mid_track):
    track_dict = {}
    
    track_dict['MetaMessage'] = []
    track_dict['Message'] = []
    track_dict['ETC'] = []
    
    for msg in mid_track:
        
        if isinstance(msg, MetaMessage):
            track_dict['MetaMessage'].append(msg)
            
        elif isinstance(msg, Message):
            track_dict['Message'].append(msg)
            
        else:
            track_dict['ETC'].append(msg)
            
    return track_dict

In [6]:
def extract_note(track_file, mid_msg):
    
    def initialize_note_ticks():
        tick_type = []
        tick_note = []
        tick_chan = []
        tick_velo = []
    
        return tick_type, tick_note, tick_chan, tick_velo
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    note_cur_time = 0
    note_start = True
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    def note_check(msg_type):
        if msg_type !='note_on' and msg_type !='note_off':
            return False
        
        else:
            return True
    
    def insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo):
        until_tick = df_note.shape[0]
                
        for note_idx in range(until_tick, note_cur_time):
            df_note.loc[note_idx, "tick"] = note_cur_time
            df_note.loc[note_idx, "count"] = len(note_tick_note)
            df_note.loc[note_idx, "msg_type"] = note_tick_type
            df_note.loc[note_idx, "msg_note"] = note_tick_note
            df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
            df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
    
    for msg in mid_msg:
        
        # note_check
        if note_check(msg.type) == False:
            continue
        
        # note_type
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        if msg.time > 0:
            
            if note_start == True:
                note_start = False
            else:
                insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)
                note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
                
            note_cur_time = msg.time + note_cur_time
            
        note_tick_type.append(msg_typ)
        note_tick_note.append(msg.note)
        note_tick_chan.append(msg.channel)
        note_tick_velo.append(msg.velocity)
            
    if len(note_tick_note) > 0:
        insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)

    df_note.to_csv(track_file)
    
    return df_note

#### >> extract_control_data_v2

In [ ]:
def extract_msg_data(track_file, mid_msg):
    
    def initialize_note_ticks():
        tick_type = []
        tick_note = []
        tick_chan = []
        tick_velo = []
    
        return tick_type, tick_note, tick_chan, tick_velo
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel', 'control', 'value'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    note_cur_time = 0
    note_start = True
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    def note_check(msg_type):
        if msg_type !='note_on' and msg_type !='note_off':
            return False
        
        else:
            return True
    
    def insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo):
        until_tick = df_note.shape[0]
                
        for note_idx in range(until_tick, note_cur_time):
            df_note.loc[note_idx, "tick"] = note_cur_time
            df_note.loc[note_idx, "count"] = len(note_tick_note)
            df_note.loc[note_idx, "msg_type"] = note_tick_type
            df_note.loc[note_idx, "msg_note"] = note_tick_note
            df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
            df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
    
    for msg in mid_msg:
        
        # note_check
        if note_check(msg.type) == False:
            continue
        
        # note_type
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        if msg.time > 0:
            
            if note_start == True:
                note_start = False
            else:
                insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)
                note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
                
            note_cur_time = msg.time + note_cur_time
            
        note_tick_type.append(msg_typ)
        note_tick_note.append(msg.note)
        note_tick_chan.append(msg.channel)
        note_tick_velo.append(msg.velocity)
            
    if len(note_tick_note) > 0:
        insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)

    df_note.to_csv(track_file)
    
    return df_note

In [8]:
def extract_data_info(file_name, mid):
    mid_info  = {}
    mid_track = []
    
    # mid_info
    mid_info['type'] = mid.type
    mid_info['ticks_per_beat'] = mid.ticks_per_beat
    mid_info['track'] = mid_track
    
    for idx, track in enumerate(mid.tracks):
        
        track_dict = distribute_msg(track)
        
        track_file_name = file_name.split(".")[0] + "_" + str(idx) + ".csv"
        df_note = extract_note(track_file_name, track_dict['Message'])
        
        print(df_note)
        mid_track.append(df_note)
        if idx==1:
            break
        
    return mid_info

input_info = extract_data_info(input_name, input_mid)
target_info = extract_data_info(target_name, target_mid)

Empty DataFrame
Columns: [msg_type, count, msg_channel, msg_note, msg_velocity, tick]
Index: []
         msg_type count msg_channel msg_note msg_velocity   tick
0       [note_on]     1         [0]     [68]         [64]    360
1       [note_on]     1         [0]     [68]         [64]    360
2       [note_on]     1         [0]     [68]         [64]    360
3       [note_on]     1         [0]     [68]         [64]    360
4       [note_on]     1         [0]     [68]         [64]    360
...           ...   ...         ...      ...          ...    ...
52795  [note_off]     1         [0]     [73]         [70]  52800
52796  [note_off]     1         [0]     [73]         [70]  52800
52797  [note_off]     1         [0]     [73]         [70]  52800
52798  [note_off]     1         [0]     [73]         [70]  52800
52799  [note_off]     1         [0]     [73]         [70]  52800

[52800 rows x 6 columns]
Empty DataFrame
Columns: [msg_type, count, msg_channel, msg_note, msg_velocity, tick]
Index: []
  

In [13]:
def extract_compression_data_info(file_name, mid):
    mid_info  = {}
    mid_track = []
    
    # mid_info
    mid_info['type'] = mid.type
    mid_info['ticks_per_beat'] = mid.ticks_per_beat
    mid_info['track'] = mid_track
    
    
    for idx, track in enumerate(mid.tracks):
        
        track_dict = distribute_msg(track)
        
        track_file_name = file_name.split(".")[0] + "_" + str(idx) + ".csv"
        df_note = pd.read_csv(track_file_name)
        df_note.drop(df_note.columns[0], axis=1, inplace=True)
        
        print(df_note)
        mid_track.append(df_note)
        if idx==1:
            break
        
    return mid_info


input_comp_info = extract_compression_data_info(input_name, input_mid)
target_comp_info = extract_compression_data_info(target_name, target_mid)

Empty DataFrame
Columns: [msg_type, count, msg_channel, msg_note, msg_velocity, tick]
Index: []
           msg_type  count msg_channel msg_note msg_velocity   tick
0       ['note_on']      1         [0]     [68]         [64]    360
1       ['note_on']      1         [0]     [68]         [64]    360
2       ['note_on']      1         [0]     [68]         [64]    360
3       ['note_on']      1         [0]     [68]         [64]    360
4       ['note_on']      1         [0]     [68]         [64]    360
...             ...    ...         ...      ...          ...    ...
52795  ['note_off']      1         [0]     [73]         [70]  52800
52796  ['note_off']      1         [0]     [73]         [70]  52800
52797  ['note_off']      1         [0]     [73]         [70]  52800
52798  ['note_off']      1         [0]     [73]         [70]  52800
52799  ['note_off']      1         [0]     [73]         [70]  52800

[52800 rows x 6 columns]
Empty DataFrame
Columns: [msg_type, count, msg_channel, msg_no

- 비교 알고리즘 다시 생각해볼 필요가 있음
    + time 기준
    + 어떤 것을 바탕으로 평가 알고리즘

In [16]:
def processing_note_on_data(input_track, target_track):
    input_notes = []
    target_notes = []
    for note_track in input_track:
        for index in range(note_track.shape[0]):
            for count in range(note_track.loc[index,'count']):
                if note_track.loc[index, 'msg_type'][count] == 'note_on':
                    input_notes.append(note_track.loc[index, 'msg_note'][count])
            
    for note_track in target_track:
        for index in range(note_track.shape[0]):
            for count in range(note_track.loc[index,'count']):
                if note_track.loc[index, 'msg_type'][count] == 'note_on':
                    target_notes.append(note_track.loc[index, 'msg_note'][count])
                
    return input_notes, target_notes

In [17]:
def calculate_note_accuracy(input_info, target_info, limit_offset=0):
    """
    인풋 MIDI와 정답 MIDI의 pitch 정확도 계산 - Yeong-Min Ko
    
    Args:
        input_midi: 인풋 MIDI 파일
        target_midi: 타겟 MIDI 파일
        
    Return:
        pitch 정확도
    """

    # 인풋 MIDI와 타겟 MIDI의 음표 정보를 추출
    input_notes, target_notes = processing_note_on_data(input_info['track'], target_info['track'])
    
    # 피치 정확도 계산
    # time 별로 기준을 세울 필요가 있음
    total_accuracy = 0
    for input_note in input_notes:
        for reference_note in target_notes:
            if abs(input_note - reference_note) < limit_offset:
                total_accuracy += 1
                break

    # 전체 음표 수와 정확한 음표 수를 이용하여 정확도 계산
    pitch_accuracy = (total_accuracy / len(target_notes)) * 100
    if pitch_accuracy > 100:
        pitch_accuracy %= 100
    
    return pitch_accuracy


In [18]:
def calculate_accuracy_level(accuracy):
    """
    음정 정확도에 따라 Level을 계산하는 함수

    Args:
        accuracy: 음정 정확도

    Return:
        계산된 Level
    """
    level = int(min(10, round(accuracy / 10)))  # 10% 단위로 Level 계산, 최대 Level은 10, 반올림 적용
    return level

In [19]:
note_accuracy = calculate_note_accuracy(input_info, target_info, 5)
note_level = calculate_accuracy_level(note_accuracy)

In [20]:
print(f'Pitch Accuracy testCase 1: {note_accuracy:.2f}% - Level {note_level}') ## track[1] 에 대해서만 함
print(note_level)

Pitch Accuracy testCase 1: 21.23% - Level 2
2


In [27]:
print(input_info['track'][1].shape[0])
print(target_info['track'][1].shape[0])

52800
52800


In [26]:
for index in range(target_info['track'][1].shape[0],input_info['track'][1].shape[0] ): ## 아마 전처리 하는 작업이 필요할 듯
    ## 너무 오래걸리는데 다른 알고리즘 생각 좀 해봐야겠음
    target_info['track'][1].loc[index, "tick"] = input_info['track'][1].loc[index, "tick"]
    target_info['track'][1].loc[index, "count"] = 1
    target_info['track'][1].loc[index, "msg_type"] = ['note_on']
    target_info['track'][1].loc[index, "msg_note"] = [0]
    target_info['track'][1].loc[index, "msg_velocity"] = [0]
    target_info['track'][1].loc[index, "msg_channel"] = [0]

In [34]:
input_df = input_info['track'][1]
target_df = target_info['track'][1]

limit_offset = 5
total_accuracy = 0

total_play_time = input_df.shape[0]

for tick in range(total_play_time):
    for input_count in range(input_df.loc[tick, 'count']):
        if input_df.loc[tick, 'msg_type'][input_count] == 'note_on':
            input_note = input_df.loc[tick, 'msg_note'][input_count]
            for target_count in range(target_df.loc[tick, 'count']):
                if target_df.loc[tick, 'msg_type'][target_count] == 'note_on':
                    target_note = target_df.loc[tick, 'msg_note'][target_count]
                    if abs(input_note - target_note) < limit_offset:
                        total_accuracy += 1
                        break
            
                
pitch_accuracy = (total_accuracy / total_play_time) * 100
if pitch_accuracy > 100:
    pitch_accuracy %= 100
    
print(total_accuracy)
print(pitch_accuracy)

12344
23.37878787878788


#### 8. 옥타브 ( note )

In [52]:
def evaluate_octave_difference(input_info, target_info):
    
    input_notes, target_notes = processing_note_on_data(input_info['track'], target_info['track'])

    # 각 곡의 옥타브 정보 추출
    input_octaves = [in_note // 12 for in_note in input_notes]
    target_octaves = [tar_note // 12 for tar_note in target_notes]
    
    # 두 곡의 옥타브 일치 여부를 평가하여 점수를 계산
    octave_matching_score = sum(1 for input_octave, target_octave in zip(input_octaves, target_octaves) if input_octave == target_octave)

    # 두 곡의 옥타브 일치 비율을 계산
    octave_matching_ratio = octave_matching_score / len(target_octaves)

    # 옥타브 일치 점수를 만점(100점) 기준으로 계산
    score = octave_matching_ratio * 100

    # 옥타브 정보 출력
    # print(f"input의 옥타브 정보: {input_octaves}")
    # print(f"target의 옥타브 정보: {target_octaves}")

    return score

def calculate_octave_level(score):
    """
    두 곡 간의 옥타브 평가 점수를 기반으로 Level을 계산하는 함수

    Arg:
        score: 현재 점수

    Returns:
        계산된 Level
    """
    score = float(score)  # 문자열에서 숫자로 변환
    level = int(min(10, round(score / 10))) # 최대 Level은 10
    return level

In [55]:
input_df = input_info['track'][1]
target_df = target_info['track'][1]

limit_offset = 5
octave_summation = 0

total_play_time = input_df.shape[0]

for tick in range(total_play_time):
    for input_count in range(input_df.loc[tick, 'count']):
        if input_df.loc[tick, 'msg_type'][input_count] == 'note_on':
            input_note = input_df.loc[tick, 'msg_note'][input_count]
            input_octave = input_note // 12
            for target_count in range(target_df.loc[tick, 'count']):
                if target_df.loc[tick, 'msg_type'][target_count] == 'note_on':
                    target_note = target_df.loc[tick, 'msg_note'][target_count]
                    target_octave = target_note // 12
                    if input_octave == target_octave:
                        octave_summation+=1


score = (octave_summation / total_play_time) * 100

print(score)
print("Octave level : ",calculate_octave_level(score))

22.68181818181818
Octave level :  2


In [53]:
score = evaluate_octave_difference(input_info, target_info)
level = calculate_octave_level(score)

# 결과 출력
print(f"{input_name}과 {target_name}의 옥타브 평가 점수: {score:.2f} / 100")
print(f"{input_name}의 옥타브 평가 레벨: {level}")

Fugue3.mid과 Fugue1.mid의 옥타브 평가 점수: 26.40 / 100
Fugue3.mid의 옥타브 평가 레벨: 3


#### 전제 조건
- track
    + 길이가 다르면 처음부터 측정 불가능
    + track 은 **왼손 / 오른손 2개**만 있다는 가정

In [ ]:
def pre_check(input_mid, target_mid):
    # track 길이 비교
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    #return input_track_len == target_track_len
    #### time 비교하는 것 짜기
    return True

### Ⅲ. Data Comparision

In [90]:
print(len(input_mid.tracks))
print(len(target_mid.tracks))

4
5


In [93]:
for idx, track in enumerate(target_mid.tracks):
    print(f"Track {idx + 1}: {track.name}")

Track 1: 
Track 2: 
Track 3: 
Track 4: 
Track 5: 


#### 11. 페달링 ( Pedaling ) // control

In [ ]:
def extract_data(input_mid, target_mid):
    input_info = {}
    target_info = {}
    
    input_info['MetaMessage']
    input_info['Message']
    
    target_info['MetaMessage']
    target_info['Message']
    
    

In [ ]:
df_info_col = ['sec', 'tick', 'msg_type', 'channel', 'note', 'velocity', 'count', 'main_vol','depth', 'pedal', 'pan', 'tempo']
df_info = pd.DataFrame(columns = df_info_col)

input_info = {MetaMessage:{}, Message:df_info }

df_tempo_col = ['tempo', 'tick']
df_tempo = pd.DataFrame(columns = df_tempo_col)

###########################
for i,track in enumerate(target_mid.tracks):
    cur_tick = 0
    
    cur_sec = 0
    
    cur_info = {'sec':0, 'tick':0, 'msg_type': "", 'channel':0, 'note':0, 
                'velocity':0, 'count':0, 'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1, 'tempo':0}
    
    for msg in track:
        
        if isinstance(msg, MetaMessage):
            
            if msg.type not in input_info[MetaMessage]:
                input_info[MetaMessage][msg.type] = msg
            
            else:
                if type(input_info[MetaMessage][msg.type]) != list:
                    input_info[MetaMessage][msg.type] = [input_info[MetaMessage][msg.type]]
                    
                input_info[MetaMessage][msg.type].append(msg)
        
        elif isinstance(msg, Message):
            print(msg)
            if msg.time > 0:
                cur_tick = cur_tick + msg.time
                
                # insert
                cur_tempo = 300000
                cur_sec = mido.tick2second(cur_tick, input_mid.ticks_per_beat, cur_tempo )
                cur_count = 1
                
                #if type(cur_note) == list:
                #    cur_count = len(cur_note)
                
                temp = [cur_info['sec'], cur_info['tick'], cur_info['msg_type'], cur_info['channel'], cur_info['note'], cur_info['velocity'], 
                        cur_info['count'],cur_info['main_vol'], cur_info['depth'], cur_info['pedal'], cur_info['pan'], cur_info['tempo']]
                
                last_tick = input_info[Message].shape[0]
                
                for idx in range(last_tick, cur_tick):
                    temp[0] = mido.tick2second(idx, input_mid.ticks_per_beat, cur_tempo )
                    temp[1] = idx
                    df_info.loc[idx] = temp
                    
                cur_info = {'sec':0, 'tick':0, 'msg_type': "", 'channel':0, 'note':0, 
                'velocity':0, 'count':0, 'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1, 'tempo':0}
            
            
            if msg.type == 'note_on' and msg.velocity == 0:
                msg_type = 'note_off'
                
            else:
                msg_type = msg.type
    
            if msg.type == 'note_on' or msg.type == 'note_off':
                if cur_info['count'] == 1:
                    cur_info['msg_type'] = [cur_info['msg_type']]
                    cur_info['channel']  = [cur_info['channel']]
                    cur_info['note']     = [cur_info['note']]
                    cur_info['velocity'] = [cur_info['velocity']]
                    
                    cur_info['msg_type'].append(msg_type)
                    cur_info['channel'].append(msg.channel)
                    cur_info['note'].append(msg.note)
                    cur_info['velocity'].append(msg.velocity)
                    
                else:
                    cur_info['msg_type'] = msg_type
                    cur_info['channel'] = msg.channel
                    cur_info['note'] = msg.note
                    cur_info['velocity'] = msg.velocity
                    
                cur_info['count']+=1
                    
            elif msg.type == 'program_change':
                print("program_change")
                
            elif msg_type == 'control_change':
                
                if msg.control == 1:
                    ctl_type = 'modulation'
                elif msg.control == 7:
                    ctl_type = 'main_vol'
                elif msg.control == 10:
                    ctl_type = 'pan'
                elif msg.control == 64:
                    ctl_type = 'pedal'
                elif msg.control >= 91 and msg.control <= 93:
                    ctl_type = 'depth'
                    
                if type(cur_info[ctl_type]) == list:
                    cur_info[ctl_type].append(msg.value)
                else:
                    if cur_info[ctl_type] == -1:
                        cur_info[ctl_type] = msg.value
                    else:
                        cur_info[ctl_type] = [cur_info[ctl_type]]
                        cur_info[ctl_type].append(msg.value)

            tempo_idx = 0
            for tempo_idx in range(df_tempo.shape[0]):
                if df_tempo.loc[tempo_idx, 'tick'] > cur_tick:
                    cur_tempo = df_tempo.loc[tempo_idx, 'tempo']
                    break
                
            if tempo_idx == df_tempo.shape[0]:
                cur_tempo = df_tempo.loc[df_tempo.shape[0]-1, 'tempo']
                
            cur_info['sec'] = mido.tick2second(cur_tick, input_mid.ticks_per_beat, cur_tempo )
            cur_info['tick'] = cur_tick
            cur_info['tempo'] = cur_tempo
                
            
        if msg.time == 118:
            break
        
    if i==0:
        tempo_tick = 0
        
        if type(input_info[MetaMessage]['set_tempo']) !=list:
            df_tempo.loc[0, 'tempo'] = input_info[MetaMessage]['set_tempo'].tempo
            df_tempo.loc[0, 'tick'] = -1
            
        else:
            for i, msg in enumerate(input_info[MetaMessage]['set_tempo']):
                tempo_tick += msg.time
            
                if i==0:
                    df_tempo.loc[i,'tempo'] = msg.tempo
                else:
                    df_tempo.loc[i,'tempo'] = msg.tempo
                    df_tempo.loc[i-1,'tick'] = tempo_tick

            df_tempo.loc[df_tempo.shape[0] -1, 'tick'] = -1
        
    if i==1:
        break

input_info[Message].replace(-1, 0, inplace=True)
        
input_info[Message].to_csv("hello.csv")
print(display(input_info[Message]))
